In [1]:
import matplotlib
import cv2
import os
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pathlib

In [3]:
data = []
labels = []

data_root = pathlib.Path("C:\\UIR\\samples\\sample_1\\cut_letters\\cut_letters")
imagePaths = list(data_root.glob('*/*'))
imagePaths = [str(path) for path in imagePaths]
random.shuffle(imagePaths)

for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    #image = cv2.resize(image,(28,28)).flatten()
    image = cv2.resize(image,(28,28))
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [4]:
data[0].shape

#image = cv2.resize(image,(28,28)).flatten()

(28, 28, 3)

In [5]:
data = np.array(data,dtype="float") / 255.0
labels = np.array(labels)

In [6]:
(trainX,testX, trainY,testY) = train_test_split(data, labels, test_size = 0.25, random_state = 42)

In [7]:
testX.shape

(21000, 28, 28, 3)

In [8]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [9]:
print('X_train shape:', trainX.shape)
print(trainX.shape[0], 'train samples')
print(testX.shape[0], 'test samples')

X_train shape: (63000, 28, 28, 3)
63000 train samples
21000 test samples


In [10]:
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 3)
num_classes = 28

if K.image_data_format() == 'channels_first':
    trainX = trainX.reshape(trainX.shape[0], 1, img_rows, img_cols)
    testX = testX.reshape(testX.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    trainX = trainX.reshape(trainX.shape[0], img_rows, img_cols, 1)
    testX = testX.reshape(testX.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

ValueError: cannot reshape array of size 148176000 into shape (63000,28,28,1)

In [11]:
EPOCHS = 50
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 3) #input_shape будет равен 28x28x3
num_classes = 28


model = Sequential()
model.add(Conv2D(filters=50,kernel_size=(5, 5),strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
model.add(Conv2D(filters=100,kernel_size=(5, 5),strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=2))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

#Компилируем модель, используя adadelta как оптимизатор и категориальную
# кросс-энтропию в качестве функции потерь. Добавляем метрики
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [12]:
#отобразим получившуюся модель
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 50)        3800      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 50)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 100)         125100    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 100)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               480300    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)              

In [15]:
from keras import callbacks
tbCallBack = callbacks.TensorBoard(log_dir="C:\\UIR\\network_01\\logs", histogram_freq = 1, write_graph = False, write_images = True)

In [16]:
H = model.fit(trainX, trainY, verbose=2, validation_data=(testX, testY), epochs=EPOCHS, batch_size=32, callbacks=[tbCallBack])

Train on 63000 samples, validate on 21000 samples
Epoch 1/50
 - 87s - loss: 0.0709 - accuracy: 0.9812 - val_loss: 0.0095 - val_accuracy: 0.9981
Epoch 2/50
 - 88s - loss: 0.0065 - accuracy: 0.9985 - val_loss: 0.0089 - val_accuracy: 0.9985
Epoch 3/50
 - 91s - loss: 0.0045 - accuracy: 0.9991 - val_loss: 0.0068 - val_accuracy: 0.9988
Epoch 4/50
 - 86s - loss: 0.0035 - accuracy: 0.9992 - val_loss: 0.0065 - val_accuracy: 0.9992
Epoch 5/50
 - 89s - loss: 0.0026 - accuracy: 0.9994 - val_loss: 0.0061 - val_accuracy: 0.9990
Epoch 6/50
 - 86s - loss: 0.0022 - accuracy: 0.9996 - val_loss: 0.0071 - val_accuracy: 0.9991
Epoch 7/50
 - 87s - loss: 0.0016 - accuracy: 0.9997 - val_loss: 0.0073 - val_accuracy: 0.9991
Epoch 8/50
 - 86s - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.0064 - val_accuracy: 0.9992
Epoch 9/50
 - 88s - loss: 9.6077e-04 - accuracy: 0.9997 - val_loss: 0.0067 - val_accuracy: 0.9992
Epoch 10/50
 - 87s - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0072 - val_accuracy: 0.9992
Epoch

In [17]:
model.save('model_01.h5')

In [33]:
f = open("C:\\UIR\\network_01\\bin_class.txt", "wb")
f.write(pickle.dumps(lb))
f.close()

In [37]:
#new_model = keras.models.load_model('my_model.h5')
#lb = pickle.loads(open("C:\\UIR\\network_01\\bin_class.txt", "rb").read()) #загружаем сохраненный бинаризатор меток

In [19]:
loss, acc = model.evaluate(testX,  testY, verbose=2)
print("Точность восстановленной модели: {:5.2f}%".format(100*acc))

Точность восстановленной модели: 99.94%


In [20]:
test_image = cv2.imread("C:\\UIR\\samples\\sample_1\\cut_letters\\extra\\4\\4955375.jpeg")
output = test_image.copy()
test_image = cv2.resize(test_image,(28,28))
test_image = test_image.astype("float") / 255.0

In [23]:
test_image = test_image.reshape((1,test_image.shape[0], test_image.shape[1],test_image.shape[2]))
test_image.shape

(1, 28, 28, 3)

In [25]:
preds = model.predict(test_image)

In [35]:
i = preds.argmax(axis=1)[0]
test_label = lb.classes_[i]

In [36]:
preds

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [40]:
print("Результат распознавания: " + test_label)

Результат распознавания: 4


In [38]:
text = "{}: {:.2f}%".format(test_label, preds[0][i] * 100)
text

'4: 100.00%'

In [1]:
#plotting training and validation loss
import matplotlib.pyplot as plt
loss = H.history['loss']
val_loss = H.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'H' is not defined